<a href="https://colab.research.google.com/github/dukei/dls-agents/blob/main/DLS_project_agents_2_agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Оценка релевантности организаций запросам на Яндекс.Картах с помощью LLM-агента

<img src="https://sun9-65.userapi.com/impg/N4y2cxlL7PauAs82tBNFOUAiNctFICWDy4Mbiw/Jiz1fb7NLWU.jpg?size=1080x1080&quality=95&sign=df2786058624d9ccac3ede4d5d056e2f&type=album" width="500" height="500" />


## Описание задачи

Необходимо создать LLM-агента, который будет оценивать релевантность организаций на Яндекс.Картах широким запросам (например, "ресторан с верандой" или "романтичный джаз-бар"). Такие запросы называются *рубричными*: пользователь здесь ищет не конкретную организацию, а идёт в Яндекс.Карты для поиска и выбора мест.

LLM-агент должен будет самостоятельно находить необходимые данные для принятия правильного решения.

Данные представлены компанией Яндекс и являются результатами асессорской разметки релевантности. Мы очень заинтересован в успешном решении задачи:)

## План работы

1. Написать сильный бейзлайн. Например, можно просто делать один запрос в LLM, возможно с добавлением в промпт размеченных примеров. Либо можно дообучить трансформер на задачу классификации.
2. Разобраться с фреймворком для реализации LLM-агентов (предлагается использовать LangGraph, оба примера в доп. материалах используют его).
3. Реализовать первую версию агента: предложить ему не принимать решение мгновенно, а ходить в поисковик для уточнения своего ответа.
4. Поработать над промптами для улучшения качества. Здесь нужно будет посмотреть, в каких запросах агент ошибается. **ПОЖАЛУЙСТА, НЕ ПОДГЛЯДЫВАЙТЕ, КАК ИМЕННО АГЕНТ ОШИБАЕТСЯ НА EVAL-МНОЖЕСТВЕ, ДЛЯ ЭТОГО ЕСТЬ TRAIN**.
5. Другие идеи (опционально):
- с помощью нейросетевого ретривала можно найти в обучающем множестве похожие размеченные примеры
- можно реализовать tool для парсинга сайта, которую агент будет вызывать

## Про платные API

Обратите внимание, что для реализации проекта потребуется самостоятельно ходить в OpenAI API (и другие платные API). На это может уйти существенная сумма (по грубой оценке ментора, 500 запусков агента на стеке OpenAI легко уложатся в 2к рублей, но риски вылезти за эту границу есть. Можно использовать и более дешевые LLM). Также поможет иностранная карта для оплаты этих сервисов, но в целом без нее можно обойтись.

Сам я для запросов в OpenAI API часто использую сервис https://vsegpt.ru/ (не реклама). При их использовании вам не нужен VPN, достаточно российской карты, а цены не сильно отличаются от цен OpenAI, но нужно оплатить подписку (400 рублей).

**О ТОМ, КАКИЕ КЛАССНЫЕ И ДЕШЕВЫЕ АЛЬТЕРНАТИВЫ ВЫ НАШЛИ, ОБЯЗАТЕЛЬНО НАПИШИТЕ В ЧАТ**

## Система оценивания

1. Реализация бейзлайна (без использования агента, но можно с использованием LLM) -- 2 балла
2. Реализация "какого-то" агента, который под капотом имеет возможность обращаться к поисковой строке для уточнения результата -- 4 балла
3. Существенно побить бейзлайн или безуспешно приложить к этому усилия и "показать", что агенты в этой задаче прироста не дают -- 4 балла
4. (опционально, за шоколадку) реализовать одну из доп. идей по улучшению агента

## Что почитать/посмотреть по теме

1. Статья, по которой можно составить какое-то представление об агенте -- https://habr.com/ru/articles/864646/
2. https://www.youtube.com/watch?v=U6LbW2IFUQw -- неплохое видео, где на практике подробно расписано, как агента можно создавать
3. Придётся самим много читать и разбираться. Делитесь идеями в чате:)

## Описание и загрузка данных

Данные: https://disk.yandex.ru/d/6d5hFHvpAZjQdw

Ваша задача -- предсказать колонку relevance, используя все остальные данные об организации. Загрузим данные и посмотрим на них

In [102]:
import requests

def download_file_from_yandex_disk(url, filename):
    base_url = "https://cloud-api.yandex.net/v1/disk/public/resources/download?public_key="
    response = requests.get(base_url + url)
    download_link = response.json()['href']
    download_response = requests.get(download_link)
    with open(filename, 'wb') as f:
        f.write(download_response.content)
    print(f"File '{filename}' downloaded successfully.")

yandex_disk_url = "https://disk.yandex.ru/d/6d5hFHvpAZjQdw"
filename = "data_final_for_dls.jsonl"
download_file_from_yandex_disk(yandex_disk_url, filename)

File 'data_final_for_dls.jsonl' downloaded successfully.


In [103]:
import pandas as pd

data = pd.read_json(path_or_buf="data_final_for_dls.jsonl", lines=True)
display(data.head())

,Text,address,name,normalized_main_rubric_name_ru,permalink,prices_summarized,relevance,reviews_summarized
0,сигары,"Москва, Дубравная улица, 34/29",Tabaccos; Магазин Tabaccos; Табаккос,Магазин табака и курительных принадлежностей,1263329400,None,1.0,"Организация занимается продажей табака, курите..."
1,кальянная спб мероприятия,"Санкт-Петербург, Большой проспект Петроградско...",PioNero; Pionero; Пицца Паста бар; Pio Nero; P...,Кафе,228111266197,PioNero предлагает разнообразные блюда итальян...,0.0,"Организация PioNero — это кафе, бар и ресторан..."
2,Эпиляция,"Московская область, Одинцово, улица Маршала Жу...",MaxiLife; Центр красоты и здоровья MaxiLife; Ц...,Стоматологическая клиника,1247255817,"Стоматологическая клиника, массажный салон и к...",1.0,"Организация занимается стоматологическими, кос..."
3,спортзал где 1 занятие бесплатно,"Москва, Страстной бульвар, 13А",Унца Унца Спорт; Unza Unza Sport,Фитнес-клуб,201938477844,Фитнес-клуб предлагает пробные занятия по разл...,0.1,Организация «Унца Унца Спорт» предоставляет ус...
4,стиральных машин,"Москва, улица Обручева, 34/63",М.Видео; M Video; M. Видео; M.Видео; Mvideo; М...,Магазин бытовой техники,1074529324,М.Видео предлагает широкий ассортимент бытовой...,1.0,Организация занимается продажей бытовой техник...


In [104]:
data['relevance'].value_counts()

relevance
1.0    15882
0.0    14509
0.1     4703
Name: count, dtype: int64

Здесь 1.0 соответствует оценке RELEVANT_PLUS, 0.1 -- оценке RELEVANT_MINUS, 0.0 -- оценке IRRELEVANT.

Ваша задача -- построить LLM-агента, который будет предсказывать релевантность.

Выделим данные для оценки качества агента. Запуск агента -- это тяжелая и потенциально дорогая операция. Поэтому eval-множество имеет размер 500. Также для простоты из eval-множества выкинуты данные с оценкой RELEVANT_MINUS. Тем не менее, вы можете использовать такие примеры для подачи примеров агенту.

**ОБРАТИТЕ ВНИМАНИЕ, ЧТО В EVAL-ДАННЫЕ НЕЛЬЗЯ ПОДГЛЯДЫВАТЬ ДЛЯ КАЛИБРОВКИ АГЕНТА!!! ДЛЯ ЭТОГО ЕСТЬ ОБУЧАЮЩИЕ ДАННЫЕ**

В качестве метрики качества мы будем использовать обычную ACCURACY, поскольку классы сбалансированы.

# Создание агента
Загрузим исправленный оценочный датасет

In [105]:
import pandas as pd
import requests

# URL для нового eval_data
eval_data_url = "https://raw.githubusercontent.com/dukei/dls-agents/refs/heads/main/data_final_for_dls_eval_new.jsonl"

# Загрузка eval_data
try:
    response = requests.get(eval_data_url)
    response.raise_for_status() # Проверяем, что запрос был успешным
    # Загружаем данные построчно, так как это jsonl
    eval_data = pd.read_json(response.text, lines=True)
    print("eval_data успешно загружен с GitHub.")
except requests.exceptions.RequestException as e:
    print(f"Ошибка при загрузке eval_data с GitHub: {e}")
    eval_data = pd.DataFrame() # Создаем пустой DataFrame в случае ошибки

# Применяем существующий фильтр для eval_data
eval_data = eval_data[eval_data["relevance"] != 0.1]

# train_data остается как раньше
train_data = data[570:]

print(f"Размер train_data: {len(train_data)}")
print(f"Размер отфильтрованного eval_data: {len(eval_data)}")


eval_data успешно загружен с GitHub.
Размер train_data: 34524
Размер отфильтрованного eval_data: 500


C:\Users\dukei\AppData\Local\Temp\ipykernel_28948\3513051366.py:12: FutureWarning: Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  eval_data = pd.read_json(response.text, lines=True)


In [106]:
train_data.head(5)

,Text,address,name,normalized_main_rubric_name_ru,permalink,prices_summarized,relevance,reviews_summarized
570,налоговая 5007,"Московская область, Королёв, улица Богомолова, 4",Налоговая служба; Межрайонная ИФНС № 2; ИФНС №...,Налоговая инспекция,1377658436,None,0.0,Организация занимается обслуживанием налогопла...
571,шугаринг Красноярск,"Красноярск, микрорайон Взлётка, улица Весны, 3",Студия красоты Дарлинг; Darling; Дарлинг,Салон красоты,1085001465,Студия красоты «Дарлинг» предоставляет услуги ...,1.0,Студия красоты «Дарлинг» предоставляет бьюти-у...
572,магазины одежды и обуви,"Мурманск, улица Карла Маркса, 8/2",Союз одежда,Магазин одежды,28785496591,None,0.1,Организация занимается продажей одежды. Отзыв ...
573,новогодняя ночь 2018 в ресторане,"Москва, Кутузовский проспект, 36А",Atlantica seafood; Коллекция; Коллекция Food &...,Ресторан,1187477091,Ресторан Atlantica предлагает разнообразные бл...,1.0,Организация занимается ресторанным бизнесом и ...
574,балетная школа в санкт петербурге 12лет с прож...,"Санкт-Петербург, Невский проспект, 35В",Русская национальная балетная школа Илзе Лиепа...,Школа танцев,172170646460,None,0.0,Организация занимается обучением балету и хоре...


In [107]:
# eval_data.to_excel("eval_data.xlsx")

## Установка библиотек

Для создания агента будем использовать Deepseek LLM, потому что у неё цена-качество самое оптимальное, Яндекс поиск, потому что оценка организаций происходит на яндекс-картах, и если информация не будет найдена, пусть Яндекс пеняет сам на себя.



In [108]:
!pip install openai langchain langchain-community langchain-openai yandex_cloud_ml_sdk

## Создаём клиент Deepseek



Будем хранить свой API ключ в окружении или секретах Colab под именем `DEEPSEEK_API_KEY` и базовый URL под именем `DEEPSEEK_BASE_URL`.


In [109]:
import os
from langchain_openai import ChatOpenAI

# Получение API ключа и базового URL: сначала из переменных окружения, затем из секретов Colab
DEEPSEEK_API_KEY = os.getenv('DEEPSEEK_API_KEY')
DEEPSEEK_BASE_URL = os.getenv('DEEPSEEK_BASE_URL')

if not DEEPSEEK_API_KEY or not DEEPSEEK_BASE_URL:
    try:
        from google.colab import userdata
        if not DEEPSEEK_API_KEY:
            DEEPSEEK_API_KEY = userdata.get('DEEPSEEK_API_KEY')
        if not DEEPSEEK_BASE_URL:
            DEEPSEEK_BASE_URL = userdata.get('DEEPSEEK_BASE_URL') # Например, "https://api.deepseek.com/v1"
    except ImportError:
        print("Предупреждение: Не удалось импортировать `google.colab.userdata`. Убедитесь, что вы в Colab или установили переменные окружения.")

if not DEEPSEEK_API_KEY:
    raise ValueError("DEEPSEEK_API_KEY не установлен ни в переменных окружения, ни в секретах Colab.")
if not DEEPSEEK_BASE_URL:
    raise ValueError("DEEPSEEK_BASE_URL не установлен ни в переменных окружения, ни в секретах Colab.")

# Initialize ChatOpenAI for Deepseek
deepseek_llm = ChatOpenAI(
    model="deepseek-chat",
    temperature=0.0, # Хотелось бы как можно более воспроизводимого рассуждения, поэтому температура 0
    openai_api_key=DEEPSEEK_API_KEY,
    openai_api_base=DEEPSEEK_BASE_URL
)


## Создание инструменов для агента


## Форматирование данных об организации

Используется для передачи агенту существующих данных об организации из датасета

In [110]:
import pandas as pd
from langchain_core.tools import BaseTool

class OrganizationDataFormatterTool(BaseTool):
    name: str = "OrganizationDataFormatter"
    description: str = (
        "Форматирует необработанные данные организации и запрос пользователя в структурированную строку для агента. "
        "Входные данные должны быть словарем с ключами: 'Text' (запрос), 'name', 'address', "
        "'normalized_main_rubric_name_ru', 'prices_summarized', 'reviews_summarized'."
    )

    def _run(self, data_row: dict) -> str:
        query = data_row.get('Text', 'Не указано')
        org_name = data_row.get('name')
        org_address = data_row.get('address')
        org_rubric = data_row.get('normalized_main_rubric_name_ru')
        org_prices = data_row.get('prices_summarized')
        org_reviews = data_row.get('reviews_summarized')

        # Handle None/NaN values
        org_name = org_name if pd.notna(org_name) else 'Не указано'
        org_address = org_address if pd.notna(org_address) else 'Не указан'
        org_rubric = org_rubric if pd.notna(org_rubric) else 'Не указана'
        org_prices = org_prices if pd.notna(org_prices) else 'Информация о ценах отсутствует.'
        org_reviews = org_reviews if pd.notna(org_reviews) else 'Отзывы отсутствуют.'

        formatted_string = f"""Запрос пользователя: "{query}"

Информация об организации:
Название: {org_name}
Адрес: {org_address}
Рубрика: {org_rubric}
Сводка цен: {org_prices}
Сводка отзывов: {org_reviews}
"""
        return formatted_string

    async def _arun(self, data_row: dict) -> str:
        return self._run(data_row)

# Instantiate the OrganizationDataFormatterTool
organization_formatter_tool = OrganizationDataFormatterTool()

print("OrganizationDataFormatterTool created and instantiated with _arun implemented.")

OrganizationDataFormatterTool created and instantiated with _arun implemented.


## Поиск в Яндекс

Создадим теперь тул для поиска в Яндексе. Данные для авторизации также берем из окружения или секретов колаба


In [111]:
import requests
import xml.etree.ElementTree as ET
from langchain_core.tools import BaseTool
import asyncio
import os
import yandex_cloud_ml_sdk as sdk
from typing import Any

class YandexSearchTool(BaseTool):
    name: str = "YandexSearch"
    search_client: Any = None
    description: str = (
        "Инструмент для выполнения веб-поиска с использованием Yandex.Cloud ML Search API. "
        "Входные данные должны быть строкой поискового запроса. "
        "Он выполняет отложенный поиск и возвращает краткое изложение результатов."
    )

    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        YC_SERVICE_SEARCH_SECRET = os.getenv('YC_SERVICE_SEARCH_SECRET')
        YC_FOLDER_ID = os.getenv('YC_FOLDER_ID')

        if not YC_SERVICE_SEARCH_SECRET or not YC_FOLDER_ID:
            try:
                from google.colab import userdata
                if not YC_SERVICE_SEARCH_SECRET:
                    YC_SERVICE_SEARCH_SECRET = userdata.get('YC_SERVICE_SEARCH_SECRET')
                if not YC_FOLDER_ID:
                    YC_FOLDER_ID = userdata.get('YC_FOLDER_ID')
            except ImportError:
                print("Предупреждение: Не удалось импортировать `google.colab.userdata`. Убедитесь, что вы в Colab или установили переменные окружения.")

        if not YC_SERVICE_SEARCH_SECRET or not YC_FOLDER_ID:
            raise ValueError("Ключи API Yandex или идентификатор папки не настроены в переменных окружения или секретах Colab. YandexSearchTool не может быть инициализирован.")

        yc_ml = sdk.AsyncYCloudML(
            auth=YC_SERVICE_SEARCH_SECRET,
            folder_id=YC_FOLDER_ID
        )
        self.search_client = yc_ml.search_api.web(
            search_type="ru",
            user_agent="LangChainAgent"
        )

    async def _arun(self, query: str) -> str:
        try:
            operation = await self.search_client.run_deferred(
                query=query,
                format="xml"
            )
            raw_xml_data = await operation.wait()

            root = ET.fromstring(raw_xml_data)

            results = []
            for doc in root.findall('.//doc'):
                title_element = doc.find('.//title')
                url_element = doc.find('.//url')
                passage_elements = doc.findall('.//passage')

                title = self.getText(title_element) if title_element is not None else 'Без заголовка'
                url = self.getText(url_element) if url_element is not None else 'Без URL'

                passages = [self.getText(p) for p in passage_elements if p.text is not None]
                summary = " ".join(passages).strip()

                result_str = f"Заголовок: {title}\nURL: {url}"
                if summary:
                    result_str += f"\nКраткое описание: {summary}"
                results.append(result_str)

            if results:
                return "\n\n".join(results)
            else:
                return "Результаты поиска по запросу не найдены."

        except Exception as e:
            return f"Произошла ошибка во время поиска Yandex: {e}"

    def _run(self, query: str) -> str:
        raise NotImplementedError("YandexSearchTool поддерживает только асинхронные операции через _arun.")

    def getText(self, root):
        return " ".join(root.itertext())


yandex_search_tool = YandexSearchTool()

print("YandexSearchTool обновлен и инициализирован с исправленным получением результатов поиска Yandex.Cloud ML SDK.")

YandexSearchTool обновлен и инициализирован с исправленным получением результатов поиска Yandex.Cloud ML SDK.


## Инструмент для посещения сайтов

После многих экспериментов оказалось, что яндекс поиск не всегда может ответить на вопрос о наличии конкретной услуги у организации. И необходимо сходить к ней на сайт. Для этого сделаем соответствующий тул.

Будем использовать не просто HTTP запрос, а загрузку соответствующего сайта браузером и получение контента получившейся страницы. Потому что многие сайты сейчас динамически создаются JavaScript. Для этого используем playwright.

Передача страницы модели в html виде перегружает её контекст. Так что извлечем только основной текст и ссылки преобразованием в markdown.

In [112]:
!pip install nest_asyncio playwright markdownify
!playwright install

Сначала поэкспериментируем, как это работает

In [113]:
import asyncio
import nest_asyncio
from playwright.async_api import async_playwright
import sys
from markdownify import markdownify as md

# Страница для загрузки
url = 'http://www.sibirskoebistro.ru'

# Чтобы работало в Windows под Anaconda пришлось зайти на StackOverflow
# https://stackoverflow.com/questions/71228742/how-to-use-the-playwright-library-in-a-jupyter-notebook-instead-of-using-a-regul

# Разрешаем вложенные циклы событий для Jupyter
nest_asyncio.apply()

async def main():
    # Запускаем Playwright
    playwright = await async_playwright().start()
    # Запускаем браузер (headless=False для видимого окна)
    browser = await playwright.chromium.launch(headless=False)
    page = await browser.new_page()

    # Пример действия
    await page.goto(url)
    print(await page.title())

    # Ждем полной загрузки страницы (например, когда нет активных сетевых запросов)
    try:
        await page.wait_for_load_state()
    except Exception as e:
        print(f"Error waiting for load state of {url}: {e}")

    # Получаем отображенный DOM в виде HTML
    html_content = await page.content()

    # Используем markdownify для извлечения содержимого в формате Markdown с включением ссылок
    markdown = md(html_content, strip=['img'])

    print("\n--- Рендеренный markdown страницы ---\n")
    print(markdown[:1000]) # Выводим первые 1000 символов для примера
    print("...")

    # Не забудьте закрыть браузер после работы
    await browser.close()
    await playwright.stop()

# Запускаем асинхронную функцию
asyncio.run(main())

Сибирское бистро

--- Рендеренный markdown страницы ---

Сибирское бистро




Томск

[+7 (3822) **325-326**](tel:+73822325326)

[Рестораны](/restorany/)[Меню](/menyu/)[Акции](/promo/)[Доставка](/dostavka/)


[Личный кабинет](lk/ "Личный кабинет")

**Войти в кабинет**

Для активации в сервисе введите   
номер Вашего телефона



+7
Ваш мобильный телефон

Код из СМС

Соглашаюсь и принимаю:

*Условия [обработки персональных данных](privacy.html)   
пользователя сервиса «Профдисконт»*

Продолжить

**Ваш регион**

[Томск](/)






[Читать подробнее](promo/15.html)

Популярные товары
-----------------

341 ₽

[Бургер Двойной бифштекс](menyu/burgery/burger-dvojnoj-bifshteks.html)

384.5478 гр.

В корзину

105 ₽

[Напиток брусничный](menyu/napitki/napitok-brusnichnyj-300ml-but-d.html)

300 гр.

В корзину

242 ₽

[Солянка мясная сборная](menyu/supy/solyanka-myasnaya-sbornaya-265-15g.html)

280 гр.

В корзину

[Смотреть меню](menyu/)

Лучшие акции
------------

[**Скидка 15% ко дню рождения**

За

Теперь создаём полноценный инструмент для загрузки страниц

In [114]:
import asyncio
import nest_asyncio
from playwright.async_api import async_playwright
from langchain_core.tools import BaseTool
from typing import Any
from markdownify import markdownify as md

# Разрешаем вложенные циклы событий для Jupyter, если еще не применено
nest_asyncio.apply()

class WebPageContentTool(BaseTool):
    name: str = "WebPageContentTool"
    description: str = (
        "Извлекает полное HTML-сосодержимое веб-страницы по заданному URL. "
        "Входные данные должны быть строкой с полным URL-адресом. "
        "Инструмент использует асинхронный браузер для рендеринга страницы и получения её содержимого."
        "Возвращает HTML-код страницы или сообщение об ошибке."
    )

    async def _arun(self, url: str) -> str:
        playwright = None
        browser = None
        try:
            playwright = await async_playwright().start()
            # Запускаем браузер в headless-режиме для большей производительности
            browser = await playwright.chromium.launch(headless=True)
            page = await browser.new_page()

            # Навигация к URL
            try:
                await page.goto(url, wait_until='load')
            except Exception as e:
                print(f"Error waiting for load state of {url}: {e}")

            # Получаем отображенный DOM в виде HTML
            html_content = await page.content()
            return md(html_content, strip=['img'])
        except Exception as e:
            return f"Ошибка при извлечении содержимого страницы {url}: {e}"
        finally:
            if browser:
                await browser.close()
            if playwright:
                await playwright.stop()

    def _run(self, url: str) -> str:
        # Синхронный вызов асинхронной части, если это необходимо
        return asyncio.run(self._arun(url))

# Инициализируем новый инструмент
web_page_content_tool = WebPageContentTool()

print("WebPageContentTool создан и готов к использованию.")

WebPageContentTool создан и готов к использованию.


## Агент

Вначале, конечно, был просто какой-то агент с яндекс-поиском. Потом на разных примерах оказалось, что он неправильно расставляет приоритеты при оценке релевантности запросов, поэтому приходилось корректировать системный промпт. И он значительно вырос. То есть, "просто какой-то агент с возможностью поиска" был промежуточным этапом, и не включен в этот ноутбук. Здесь сразу предоставлен агент с системным промтом, уточняющий контекст задачи и содержащий более детальные инструкции для LLM.

Также сделаем сразу два агента - с инструментом хождения по сайтам и без него.



In [115]:
from langchain_classic.agents import AgentExecutor, create_openai_tools_agent
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

# 1. Создаем список доступных инструментов
tools = [yandex_search_tool, web_page_content_tool, organization_formatter_tool]
tools_search_only = [yandex_search_tool, organization_formatter_tool]

# 2. Определяем промпт агента
# Промпт инструктирует агента о его задаче, доступных инструментах и ожидаемом формате вывода.
agent_prompt = ChatPromptTemplate.from_messages([
    ("system", "Ты опытный ассистент, который оценивает релевантность организации запросу пользователя. "
               "Ты должен использовать доступные инструменты для сбора информации и форматирования данных. "
               "Твоя конечная цель — В ПОСЛЕДНЕЙ СТРОКЕ ответа вернуть только одно число: 1.0 (полностью релевантно, важные требования к организации удовлетворены, "
               "второстепенные удовлетворены или после поиска не найдено информации, что они не удовлетворены), "
               "0.1 (частично релевантно, важные требования к организации в запросе пользователя удовлетворены, второстепенные не удовлетворены) "
               "или 0.0 (нерелевантно, хотя бы одно важное требование не удовлетворено). "
               "Запрос пользователя следует рассматривать в контексте поиска организации на карте, "
               "то есть, указание в запросе неточного адреса можно трактовать как поиск организации поблизости от этого адреса. "
               "Имей в виду, что если в запросе пользователя есть указание на конкретную организацию, например, по её названию, адресу, номеру, "
               "то организация будет релевантна запросу, только если она именно такая. "
               "Если пользователь ожидает от организации какой-то услуги, но явно она в описании организации не представлена, "
               "попробуй найти эту информацию с помощью tools. "
               "Помни, что главный источник данных об услугах, которые представляет организация, её официальный сайт. В случае сомнений, "
               "попытайся найти её сайт и прочитать его с помощью инструмента WebPageContentTool. "
               "Если пользователя интересует определенная услуга, но она не присутствует на официальном сайте, считай, что услуги нет! "
               "Учитывай, что пользователь не всегда правильно формулирует запрос, некоторые требования могут быть противоречивы или "
               "не отражать реальное пожелание пользователя. Поэтому проявляй гибкость рассуждений. "
               "Обязательно используй tool `OrganizationDataFormatter` для форматирования входных данных."),
    MessagesPlaceholder("chat_history"),
    ("user", "{input}"),
    MessagesPlaceholder("agent_scratchpad"),
])

agent_prompt_search_only = ChatPromptTemplate.from_messages([
    ("system", "Ты опытный ассистент, который оценивает релевантность организации запросу пользователя. "
               "Ты должен использовать доступные инструменты для сбора информации и форматирования данных. "
               "Твоя конечная цель — В ПОСЛЕДНЕЙ СТРОКЕ ответа вернуть только одно число: 1.0 (полностью релевантно, важные требования к организации удовлетворены, "
               "второстепенные удовлетворены или после поиска не найдено информации, что они не удовлетворены), "
               "0.1 (частично релевантно, важные требования к организации в запросе пользователя удовлетворены, второстепенные не удовлетворены) "
               "или 0.0 (нерелевантно, хотя бы одно важное требование не удовлетворено). "
               "Запрос пользователя следует рассматривать в контексте поиска организации на карте, "
               "то есть, указание в запросе неточного адреса можно трактовать как поиск организации поблизости от этого адреса. "
               "Имей в виду, что если в запросе пользователя есть указание на конкретную организацию, например, по её названию, адресу, номеру, "
               "то организация будет релевантна запросу, только если она именно такая. "
               "Если пользователь ожидает от организации какой-то услуги, но явно она в описании организации не представлена, "
               "попробуй найти эту информацию с помощью tools. "
               "Учитывай, что пользователь не всегда правильно формулирует запрос, некоторые требования могут быть противоречивы или "
               "не отражать реальное пожелание пользователя. Поэтому проявляй гибкость рассуждений. "
               "Обязательно используй tool `OrganizationDataFormatter` для форматирования входных данных."),
    MessagesPlaceholder("chat_history"),
    ("user", "{input}"),
    MessagesPlaceholder("agent_scratchpad"),
])

# 3. Инициализируем агента
# Используем create_openai_tools_agent, потому что deepseek_llm совместим с OpenAI API.
agent = create_openai_tools_agent(
    llm=deepseek_llm,
    tools=tools,
    prompt=agent_prompt,
)

agent_search_only = create_openai_tools_agent(
    llm=deepseek_llm,
    tools=tools_search_only,
    prompt=agent_prompt_search_only,
)

# 4. Создаем AgentExecutor
# Этот исполнитель будет запускать агента с определенными инструментами и промптом.
agent_executor = AgentExecutor(
    agent=agent,
    tools=tools,
    verbose=True,
    handle_parsing_errors=True # Обрабатывает случаи, когда вывод LLM не является допустимым использованием инструмента
)

agent_executor_search_only = AgentExecutor(
    agent=agent_search_only,
    tools=tools_search_only,
    verbose=True,
    handle_parsing_errors=True # Обрабатывает случаи, когда вывод LLM не является допустимым использованием инструмента
)

print("LangChain агент и AgentExecutor инициализированы.")

LangChain агент и AgentExecutor инициализированы.


## Функция асинхронного запуска агента

Агент с кучей инструментов думает довольно долго, поэтому для запуска его на датасете неплохо бы вызывать много агентов одновременно. Поэтому сделаем асинхронную функцию выполнения агента

In [116]:
import asyncio
import re # Импорт модуля регулярных выражений

async def get_agent_prediction_async(agent_executor, data_row: dict) -> tuple[float, str]:
    """
    Асинхронно получает предсказание релевантности от LangChain агента для заданной организации.

    Args:
        data_row (dict): Словарь, содержащий данные об организации (например, 'Text', 'name', 'address' и т.д.).

    Returns:
        tuple[float, str]: Кортеж, содержащий предсказанную оценку релевантности (1.0, 0.1 или 0.0) и полную строку рассуждений.
    """
    try:
        # Форматируем входные данные с использованием пользовательского инструмента
        formatted_input = organization_formatter_tool._run(data_row)

        # Асинхронно вызываем исполнителя агента
        result = await agent_executor.ainvoke({"input": formatted_input, "chat_history": []})

        # Извлекаем и парсим вывод агента
        prediction_full_output = result.get('output', '').strip()

        # Гибко извлекаем ответ LLM. Несмотря на инструкции, она не всегда возвращает только число
        prediction_lines = [line.strip() for line in prediction_full_output.split('\n') if line.strip()]
        last_line_output = prediction_lines[-1] if prediction_lines else ''

        valid_scores_str = ['0.0', '0.1', '1.0']
        found_predictions_unique = set() # Используем множество для хранения уникальных предсказаний

        for score_str in valid_scores_str:
            # Используем регулярное выражение для поиска оценки как целого слова, учитывая потенциальную разметку (например, **0.1**)
            # и убеждаемся, что она не является частью большего числа (например, '10.0')
            matches = re.findall(r'\b' + re.escape(score_str) + r'\b', last_line_output)
            if matches:
                found_predictions_unique.add(float(score_str))

        prediction_float = -1.0 # Значение по умолчанию для ошибок или неоднозначных случаев

        if len(found_predictions_unique) == 1:
            # Если найдена только одна уникальная валидная оценка, используем ее.
            prediction_float = list(found_predictions_unique)[0]
        elif len(found_predictions_unique) > 1:
            # Если найдено несколько уникальных валидных оценок, это неоднозначно.
            print(f"Предупреждение: Агент вернул несколько уникальных валидных оценок релевантности в последней строке: {found_predictions_unique}. Возвращаем -1.0 из-за неоднозначности.")
        else:
            # Если валидные оценки не найдены нигде в последней строке.
            print(f"Предупреждение: Агент не вернул валидных оценок релевантности (0.0, 0.1, 1.0) в последней строке. Вывод последней строки: '{last_line_output}'. Возвращаем -1.0.")
        # --- Конец измененной логики парсинга ---

        # Возвращаем распарсенное число с плавающей точкой и полный вывод/рассуждения
        return prediction_float, prediction_full_output

    except Exception as e:
        print(f"Ошибка во время предсказания агентом: {e}. Возвращаем -1.0.")
        return -1.0, f"Ошибка: {e}"

print("Определена функция асинхронного предсказания агента с исправленным парсингом вывода и возвратом рассуждений.")

Определена функция асинхронного предсказания агента с исправленным парсингом вывода и возвратом рассуждений.


Проверим, как это работает



In [88]:
import asyncio

# Возьмем пример поиска пельменной
first_train_example = train_data.iloc[27]

# Запускаем агента
predicted_relevance, agent_reasoning = await get_agent_prediction_async(agent_executor_search_only, first_train_example)

print(f"Предсказанная релевантность для первого примера из train_data: {predicted_relevance}")
print(f"Истинная релевантность: {first_train_example['relevance']}")

#print("\n--- Рассуждения агента ---")
#print(agent_reasoning)



> Entering new AgentExecutor chain...

Invoking: `OrganizationDataFormatter` with `{'data_row': {'Text': 'пельменная', 'name': 'Сибирское бистро; Sibirskoe bistro; Кафе-кондитерская Сладкоежка; Сибирское бистро, кафе; Сладкоежка; Верхнее Бистро', 'address': 'Томск, Ново-Соборная площадь, 2', 'normalized_main_rubric_name_ru': 'Быстрое питание', 'prices_summarized': 'Кафе «Сибирское бистро» предлагает широкий ассортимент блюд быстрого питания, включая салаты, супы, чебуречки, шаурму, бургеры, а также десерты и напитки | драники с мясом | салат Цезарь | шаурма классическая | бульон куриный | бургер двойной бифштекс | плов с куриным филе | пирожное картошка | морс брусничный', 'reviews_summarized': 'Организация занимается предоставлением услуг кафе, доставкой еды и обедов, а также изготовлением тортов на заказ. Отзывы смешанные: есть положительные оценки вкуса блюд и обслуживания, но также много критики по поводу качества еды, высоких цен и проблем с гигиеной. Хвалят: вкус некоторых блюд

Чисто поисковый агент решил по косвенным данным поиска решил, что пельмени в этом ресторане всё-таки есть. Но я лично убедился, что в меню на официальном сайте пельменей нет. Посмотрим, что выдаст агент с возможностью самостоятельно ходить на сайт.

In [89]:
# Возьмем пример поиска пельменной
first_train_example = train_data.iloc[27]

# Запускаем агента с хождением по сайтам
predicted_relevance, agent_reasoning = await get_agent_prediction_async(agent_executor, first_train_example)

print(f"Предсказанная релевантность для первого примера из train_data: {predicted_relevance}")
print(f"Истинная релевантность: {first_train_example['relevance']}")




> Entering new AgentExecutor chain...

Invoking: `OrganizationDataFormatter` with `{'data_row': {'Text': 'пельменная', 'name': 'Сибирское бистро; Sibirskoe bistro; Кафе-кондитерская Сладкоежка; Сибирское бистро, кафе; Сладкоежка; Верхнее Бистро', 'address': 'Томск, Ново-Соборная площадь, 2', 'normalized_main_rubric_name_ru': 'Быстрое питание', 'prices_summarized': 'Кафе «Сибирское бистро» предлагает широкий ассортимент блюд быстрого питания, включая салаты, супы, чебуречки, шаурму, бургеры, а также десерты и напитки | драники с мясом | салат Цезарь | шаурма классическая | бульон куриный | бургер двойной бифштекс | плов с куриным филе | пирожное картошка | морс брусничный', 'reviews_summarized': 'Организация занимается предоставлением услуг кафе, доставкой еды и обедов, а также изготовлением тортов на заказ. Отзывы смешанные: есть положительные оценки вкуса блюд и обслуживания, но также много критики по поводу качества еды, высоких цен и проблем с гигиеной. Хвалят: вкус некоторых блюд

Как видно, агент стал гораздо умнее. Несмотря на то, что Яндекс поиск и Яндекс афиша пытались агента запутать, он всё же сходил на официальный сайт и проверил информацию в меню. И на этот раз выдал правильный ответ.

Эта пельменная только один из ярких примеров, как самостоятельная проверка первоисточника помогает агенту принять правильное решение. А таких было много в тренировочном датасете.

## Запуск агента на датасете

Для удобства отладки лучше запустить агента сразу на каком-то подмножестве датасета. И потом аккуратно в табличном виде пройтись и оценить качество его работы.

Эта операция проделывалась много раз, в результате системный промт обрастал полезными уточнениями, а набор инструментов расширялся.



In [91]:
subset_for_agent_testing = train_data.iloc[80:100].copy()
print(f"Subset created with {len(subset_for_agent_testing)} examples.")
display(subset_for_agent_testing.head())

Subset created with 20 examples.


,Text,address,name,normalized_main_rubric_name_ru,permalink,prices_summarized,relevance,reviews_summarized
650,ударно-волновая терапия в екатеринбурге адреса...,"Свердловская область, Екатеринбург, улица Шейн...",Evolife; Эволайф; Медицинский центр лечения по...,"Медцентр , клиника",112616220470,"Evolife предлагает консультации специалистов, ...",0.0,Организация занимается медицинской реабилитаци...
651,садовый инвентарь магазин,"Липецк, улица Космонавтов, 98",Порядок; Poryadok,Товары для дома,48154757579,"Магазин «Порядок» предлагает товары для дома, ...",1.0,Организация занимается продажей товаров для до...
652,китайский ресторан на беляево,"Москва, улица Миклухо-Маклая, 15",Тяньфу; Китайский ресторан Тянь Фу,Ресторан,236131839527,None,1.0,Организация занимается подачей традиционной ки...
653,пудра плитка церсанит,"Челябинск, Копейское шоссе, 64",Лемана Про; Lemana Pro; Leroy Merlin; Леруа Ме...,Строительный магазин,1464004829,Лемана Про предлагает широкий ассортимент това...,0.0,Организация «Лемана Про» занимается продажей т...
654,прием ежевика гребенчатого дозировка,"Ростовская область, Неклиновский район, село Н...",Ezhevikatv.ru; Ezhevika; Ezhevikatv,Питомник растений,113772510470,None,0.0,Организация занимается продажей саженцев ежеви...


Теперь на отобранных примерах запускаем агента


In [117]:
# Асинхронно получаем предсказания агента на заданном датасете, учитывая
# одновременный запуск не более concurrency агентов
# Запускаем лучшего агента со всеми инструментами
async def get_all_predictions(data_subset, concurrency=50):
    tasks = []
    semaphore = asyncio.Semaphore(concurrency)

    async def process_row(row):
        async with semaphore:
            return await get_agent_prediction_async(agent_executor, row)

    for index, row in data_subset.iterrows():
        tasks.append(process_row(row))
    return await asyncio.gather(*tasks)

In [92]:
predictions_list = []
reasoning_list = []

# Запуск агента на датасете
results = await get_all_predictions(subset_for_agent_testing, concurrency=50)

for pred, reason in results:
    predictions_list.append(pred)
    reasoning_list.append(reason)

# Добавляем ответ агента и обоснование в таблицу
subset_for_agent_testing['predicted_relevance'] = predictions_list
subset_for_agent_testing['agent_reasoning'] = reasoning_list

print("Predictions and reasoning obtained for the subset with concurrency limit.")

Выходные данные были обрезаны до нескольких последних строк (5000).
Краткое описание: От  Сочи   до  хутора  Джанхот , сколько времени занимает путь с учётом пробок и без, подробное описание маршрута по платным и бесплатным дорогам на легковом или грузовом автомобиле смотрите в Яндекс Картах.

Заголовок: Расстояние  между хутором  Джанхот  и поселком  Кудепста
URL: https://stranagruzov.ru/Geo/Way/!/?FromCity=%D1%85%D1%83%D1%82%D0%BE%D1%80%20%D0%94%D0%B6%D0%B0%D0%BD%D1%85%D0%BE%D1%82&ToCity=%D0%BF%D0%BE%D1%81%D0%B5%D0%BB%D0%BE%D0%BA%20%D0%9A%D1%83%D0%B4%D0%B5%D0%BF%D1%81%D1%82%D0%B0

Заголовок: Картинки по запросу "расстояние Кудепста Сочи до Джанхот Геленджик км"
URL: https://yandex.ru/images/search?text=%D1%80%D0%B0%D1%81%D1%81%D1%82%D0%BE%D1%8F%D0%BD%D0%B8%D0%B5+%D0%9A%D1%83%D0%B4%D0%B5%D0%BF%D1%81%D1%82%D0%B0+%D0%A1%D0%BE%D1%87%D0%B8+%D0%B4%D0%BE+%D0%94%D0%B6%D0%B0%D0%BD%D1%85%D0%BE%D1%82+%D0%93%D0%B5%D0%BB%D0%B5%D0%BD%D0%B4%D0%B6%D0%B8%D0%BA+%D0%BA%D0%BC
Краткое описание: Картинки 

In [93]:
subset_for_agent_testing.head(5)

,Text,address,name,normalized_main_rubric_name_ru,permalink,prices_summarized,relevance,reviews_summarized,predicted_relevance,agent_reasoning
650,ударно-волновая терапия в екатеринбурге адреса...,"Свердловская область, Екатеринбург, улица Шейн...",Evolife; Эволайф; Медицинский центр лечения по...,"Медцентр , клиника",112616220470,"Evolife предлагает консультации специалистов, ...",0.0,Организация занимается медицинской реабилитаци...,1.0,Отлично! На официальном сайте я нашел конкретн...
651,садовый инвентарь магазин,"Липецк, улица Космонавтов, 98",Порядок; Poryadok,Товары для дома,48154757579,"Магазин «Порядок» предлагает товары для дома, ...",1.0,Организация занимается продажей товаров для до...,1.0,Теперь у меня есть полная информация. Страница...
652,китайский ресторан на беляево,"Москва, улица Миклухо-Маклая, 15",Тяньфу; Китайский ресторан Тянь Фу,Ресторан,236131839527,None,1.0,Организация занимается подачей традиционной ки...,0.1,Этот сайт tunfu.ru относится к другому рестора...
653,пудра плитка церсанит,"Челябинск, Копейское шоссе, 64",Лемана Про; Lemana Pro; Leroy Merlin; Леруа Ме...,Строительный магазин,1464004829,Лемана Про предлагает широкий ассортимент това...,0.0,Организация «Лемана Про» занимается продажей т...,1.0,"Отлично! Я нашел конкретный продукт - ""Керамич..."
654,прием ежевика гребенчатого дозировка,"Ростовская область, Неклиновский район, село Н...",Ezhevikatv.ru; Ezhevika; Ezhevikatv,Питомник растений,113772510470,None,0.0,Организация занимается продажей саженцев ежеви...,0.0,Теперь давайте проанализируем ситуацию:\n\n1. ...


Теперь проверим, в чем ошибается агент в случаях, когда его оценка расходится с референсной. Например, рассмотрим запрос `ударно-волновая терапия в екатеринбурге адреса и цены` и организацию **Медицинский центр лечения позвоночника и суставов Эволайф**. Организация помечена как нерелевантная запросу, агент считает наоборот. Почему же он так решил? Он это объясняет следующим образом:


> Отлично! На официальном сайте я нашел конкретную информацию о цене на ударно-волновую терапию. На странице указано:

> **УДАРНО – ВОЛНОВАЯ ТЕРАПИЯ**
Радиальная УВТ (ударно-волновая терапия)-1зона на аппарате Storz Medical MP 200 (Швейцария) A22.30.015 - **2500 рублей**

> Теперь у меня есть полная информация для оценки релевантности:

> **Анализ требований пользователя:**
> 1. **Ударно-волновая терапия** - ✅ предоставляется организацией
2. **В Екатеринбурге** - ✅ организация находится в Екатеринбурге
3. **Адреса** - ✅ адрес предоставлен: ул. Шейнкмана, 86
4. **Цены** - ✅ цена указана на официальном сайте: 2500 рублей за 1 зону

> Все требования пользователя полностью удовлетворены. Пользователь искал медицинский центр в Екатеринбурге, который предоставляет услугу ударно-волновой терапии, с указанием адреса и цен. Организация Evolife полностью соответствует всем этим критериям.

> 1.0

И ведь он прав! На официальном сайте есть эта услуга, даже с ценой. Да и все остальные элементы запроса удовлетворены. Видимо, ошибка в разметке датасета.

Посмотрим другой пример, `какая поликлиника дежурная по василеостровскому району` для организации **КДП № 1; Гастроэнтерологическое отделение; Диагностическая поликлиника № 1; КДП № 1, Приморский район;**. На этот раз пример размечен как полностью релевантный, агент, опять же, считает наоборот. Его обоснование:

> Теперь у меня есть вся необходимая информация для анализа. Давайте подведем итоги:

> **Анализ запроса пользователя:**
Пользователь спрашивает: "какая поликлиника дежурная по василеостровскому району". Это означает, что пользователь ищет:
> 1. Дежурную поликлинику (работающую в нерабочее время, возможно круглосуточно или в выходные)
2. В Василеостровском районе Санкт-Петербурга

> **Анализ организации:**
> 1. **Название:** Консультативно-диагностическая поликлиника № 1
2. **Адрес:** Санкт-Петербург, Приморский проспект, 3
3. **Район:** Из официального сайта и поиска видно, что это поликлиника **Приморского района**, а не Василеостровского
4. **График работы:** Согласно официальному сайту - понедельник-пятница: 08.00 — 20.00, суббота, воскресенье: выходной
5. **Дежурная ли:** Нет информации о том, что это дежурная поликлиника. Обычно дежурные поликлиники работают в выходные и праздничные дни, имеют круглосуточный режим или специальный график дежурств.

> **Ключевые несоответствия:**
> 1. **Район:** Пользователь ищет поликлинику в Василеостровском районе, а данная организация находится в Приморском районе.
2. **Дежурность:** Пользователь ищет дежурную поликлинику, а данная организация работает только в будни с 08:00 до 20:00, выходные - выходной.
3. **Тип услуги:** Дежурные поликлиники обычно предоставляют экстренную медицинскую помощь в нерабочее время, что не соответствует профилю консультативно-диагностической поликлиники.

> **Важные требования пользователя:**
> 1. ✅ Поликлиника - да, это поликлиника
2. ❌ Дежурная - нет, не дежурная (работает только в будни, выходные - выходной)
3. ❌ Василеостровский район - нет, находится в Приморском районе

> Поскольку хотя бы одно важное требование (район) не удовлетворено, организация не релевантна запросу пользователя.

> 0.0

И опять же, агент полностью прав! Это уже определяется несоответствием района в запросе пользователя расположению организации. Хотя он дополнительно ещё нашел, что поликлиника не дежурная.

Похоже, разметка датасета была сделана какими-то средствами машинного обучения без использования человека или LLM. В результате, агенты на основе LLM оценивают релевантность лучше.

В этом случае вопрос, а стоит ли тогда стремиться к высокому скору? Или это будет как высокая оценка в художественном ВУЗе - если ты получил высокую оценку, то ты мало чем отличаешься от своих учителей. А значит, не привнесешь ничего нового в искусство...


# Финальная оценка
Посчитаем теперь точность нашего агента на оценочном датасете

In [118]:
subset_for_agent_testing = eval_data.copy()
print(f"Evaluation subset created with {len(subset_for_agent_testing)} examples.")
display(subset_for_agent_testing.head())

Evaluation subset created with 500 examples.


,Text,address,name,normalized_main_rubric_name_ru,permalink,prices_summarized,relevance,reviews_summarized,relevance_new
0,сигары,"Москва, Дубравная улица, 34/29",Tabaccos; Магазин Tabaccos; Табаккос,Магазин табака и курительных принадлежностей,1263329400,None,1.0,"Организация занимается продажей табака, курите...",1.0
1,кальянная спб мероприятия,"Санкт-Петербург, Большой проспект Петроградско...",PioNero; Pionero; Пицца Паста бар; Pio Nero; P...,Кафе,228111266197,PioNero предлагает разнообразные блюда итальян...,0.0,"Организация PioNero — это кафе, бар и ресторан...",0.0
2,Эпиляция,"Московская область, Одинцово, улица Маршала Жу...",MaxiLife; Центр красоты и здоровья MaxiLife; Ц...,Стоматологическая клиника,1247255817,"Стоматологическая клиника, массажный салон и к...",1.0,"Организация занимается стоматологическими, кос...",1.0
4,стиральных машин,"Москва, улица Обручева, 34/63",М.Видео; M Video; M. Видео; M.Видео; Mvideo; М...,Магазин бытовой техники,1074529324,М.Видео предлагает широкий ассортимент бытовой...,1.0,Организация занимается продажей бытовой техник...,1.0
5,сеть быстрого питания,"Санкт-Петербург, 1-я Красноармейская улица, 15",Rostic's; KFC; Ресторан быстрого питания KFC,Быстрое питание,1219173871,Rostic's предлагает различные наборы быстрого ...,1.0,"Организация занимается быстрым питанием, предо...",1.0


In [119]:
predictions_list = []
reasoning_list = []

# Запуск агента на датасете
results = await get_all_predictions(subset_for_agent_testing, concurrency=50)

for pred, reason in results:
    predictions_list.append(pred)
    reasoning_list.append(reason)

# Добавляем ответ агента и обоснование в таблицу
subset_for_agent_testing['predicted_relevance'] = predictions_list
subset_for_agent_testing['agent_reasoning'] = reasoning_list

print("Predictions and reasoning obtained for the subset with concurrency limit.")

Выходные данные были обрезаны до нескольких последних строк (5000).

 Дополнительные общеразвивающие программы в области физической культуры и спорта:

1. Дополнительная образовательная общеразвивающая программа по виду спорта «Бодибилдинг» — 50 человек
2. Дополнительная образовательная общеразвивающая программа по виду спорта «Бокс» — **0 человек**
3. Дополнительная образовательная общеразвивающая программа по виду спорта «Волейбол» — **200 человек**
4. Дополнительная образовательная общеразвивающая программа по виду спорта «Дзюдо» — **150 человек**
5. Дополнительная образовательная общеразвивающая программа по виду спорта «Мини-футбол» — **175 человек**
6. Дополнительная образовательная общеразвивающая программа по виду спорта «Плавание» — **250 человек**
7. Дополнительная образовательная общеразвивающая программа обучения плаванию детей-инвалидов — **35 человек**
8. Дополнительная образовательная общеразвивающая программа по виду спорта «Самбо» — **49 человек**
9. Дополнительная обр

In [132]:
import pandas as pd
from sklearn.metrics import accuracy_score

# Отфильтровываем записи, где predicted_relevance было -1 (указывает на ошибку или нечисловой вывод)
# Этот отфильтрованный DataFrame используется для расчета точности на основе допустимых предсказаний.
# Явно создаем копию, чтобы избежать SettingWithCopyWarning
filtered_predictions = subset_for_agent_testing[subset_for_agent_testing['predicted_relevance'] != -1.0].copy()
print(f"Encountered {len(subset_for_agent_testing)-len(filtered_predictions)} errors")

# Добавляем новую колонку 'predicted_relevance01' в DataFrame.
# Поскольку у нас в валидационном датасете только два класса, сделаем здесь тоже только два
filtered_predictions['predicted_relevance01'] = filtered_predictions['predicted_relevance'].apply(lambda x: 0.0 if x == 0.0 else 1.0)

# Преобразуем 'relevance' и 'predicted_relevance' в строковый тип для расчета точности

# Сначала посчитаем точность на старой relevance, чтобы сравнить с бейзлайном
y_true = filtered_predictions['relevance'].astype(str)
y_pred = filtered_predictions['predicted_relevance'].astype(str)
accuracy_old = accuracy_score(y_true, y_pred)

# Теперь посчитаем точность на исправленном relevance
y_true = filtered_predictions['relevance_new'].astype(str)
y_pred = filtered_predictions['predicted_relevance'].astype(str)
accuracy = accuracy_score(y_true, y_pred)

# И наконец, поскольку в оценочном датасете всего два класса, улучшим (возможно) метрику, сведя частичную релевантность к полной
y_true = filtered_predictions['relevance_new'].astype(str)
y_pred = filtered_predictions['predicted_relevance01'].astype(str)
accuracy01 = accuracy_score(y_true, y_pred)

# Сохраняем результаты полного DataFrame (включая новую колонку) в CSV-файл
output_filename = "agent_eval_predictions_with_reasoning.csv"
subset_for_agent_testing.to_csv(output_filename, index=False)

print(f"Результаты сохранены в файл: {output_filename}")
print(f"Рассчитанная точность: {accuracy:.4f} (старая: {accuracy_old:.4f}, бинарная: {accuracy01:.4f})")

# Отображаем первые несколько строк DataFrame с предсказаниями и рассуждениями
display(subset_for_agent_testing.head())

Encountered 3 errors
Результаты сохранены в файл: agent_eval_predictions_with_reasoning.csv
Рассчитанная точность: 0.7304 (старая: 0.6197, бинарная: 0.7586)


,Text,address,name,normalized_main_rubric_name_ru,permalink,prices_summarized,relevance,reviews_summarized,relevance_new,predicted_relevance,agent_reasoning
0,сигары,"Москва, Дубравная улица, 34/29",Tabaccos; Магазин Tabaccos; Табаккос,Магазин табака и курительных принадлежностей,1263329400,None,1.0,"Организация занимается продажей табака, курите...",1.0,0.1,Отлично! На официальном сайте магазина Tabacco...
1,кальянная спб мероприятия,"Санкт-Петербург, Большой проспект Петроградско...",PioNero; Pionero; Пицца Паста бар; Pio Nero; P...,Кафе,228111266197,PioNero предлагает разнообразные блюда итальян...,0.0,"Организация PioNero — это кафе, бар и ресторан...",0.0,0.0,Результаты поиска не показывают связи PioNero ...
2,Эпиляция,"Московская область, Одинцово, улица Маршала Жу...",MaxiLife; Центр красоты и здоровья MaxiLife; Ц...,Стоматологическая клиника,1247255817,"Стоматологическая клиника, массажный салон и к...",1.0,"Организация занимается стоматологическими, кос...",1.0,1.0,"Отлично! На странице с ценами я нашел раздел ""..."
4,стиральных машин,"Москва, улица Обручева, 34/63",М.Видео; M Video; M. Видео; M.Видео; Mvideo; М...,Магазин бытовой техники,1074529324,М.Видео предлагает широкий ассортимент бытовой...,1.0,Организация занимается продажей бытовой техник...,1.0,1.0,Сайт заблокировал доступ из-за подозрения в ав...
5,сеть быстрого питания,"Санкт-Петербург, 1-я Красноармейская улица, 15",Rostic's; KFC; Ресторан быстрого питания KFC,Быстрое питание,1219173871,Rostic's предлагает различные наборы быстрого ...,1.0,"Организация занимается быстрым питанием, предо...",1.0,1.0,"Теперь у меня есть четкое подтверждение, что R..."


В процессе работы агента из 500 запусков случилось 3 ошибки (не удалось узнать ответ LLM). Эти примеры можно было бы перезапустить, но 3 ошибки на 500 запусков это менее 1%, так что на точность, вероятно, не повлияет.

По сравнению с бейзлайном результат существенно улучшен - 0.61 против [0.45](https://github.com/dukei/dls-agents/blob/main/DLS-project-agents-1_baseline.ipynb). Точность на исправленно оценочном датасете ещё выше - 0.73, что означает, что исправлялся он в правильную сторону. При этом, если мы редуцируем число классов до двух (частичную релевантность заменим на полную), то точность ещё вырастет до 0.75.

Несмотря на то, что точность существенно улучшилась, что подтверждает превосходство агента над голой LLM, интересно посмотреть, преследуя чисто академический интерес, а в чем же агент ошибался (и ошибался ли) на оценочном датасете.

## Разбор полетов

Валидационная выборка с ответам агента сохранена в github: https://github.com/dukei/dls-agents/blob/main/agent_eval_predictions_with_reasoning.csv, её можно удобно просмотреть онлайн здесь: https://jumpshare.com/s/WZt6JKjTTWcWZ0PwnxKS

Пойдем прям по порядку. Первое расхождение `сигары` для **Магазин Tabaccos**. Агент оценил как 0.1, референсное значение 1.0. Как он это объясняет?

>Отлично! На официальном сайте магазина Tabaccos в разделе "Наш ассортимент" явно указано, что они продают "Сигары". Это прямое подтверждение того, что магазин предлагает именно тот товар, который ищет пользователь.

>Теперь проанализирую ситуацию:

>1. **Запрос пользователя**: "сигары" - пользователь ищет сигары
2. **Организация**: Tabaccos - магазин табака и курительных принадлежностей
3. **Рубрика**: "Магазин табака и курительных принадлежностей" - соответствует тематике
4. **Официальный сайт**: Прямо указывает, что в ассортименте есть "Сигары"
5. **Отзывы**: В отзывах есть жалобы на отсутствие сигарет, но это может относиться к конкретным моментам или конкретным маркам сигарет, а не к сигарам вообще

>Важный момент: в отзывах указано "Жалуются на отсутствие сигарет" (пункт 1), но это может означать отсутствие обычных сигарет, а не сигар. Сигары и сигареты - это разные товары. На официальном сайте четко указано, что сигары входят в ассортимент.

>Также стоит отметить, что магазин специализируется на табачных изделиях и имеет широкий ассортимент, включая сигары, что подтверждается официальным сайтом.

>**Вывод**: Организация полностью релевантна запросу пользователя, так как:
- Это магазин табачных изделий
- На официальном сайте прямо указано, что они продают сигары
- Пользователь ищет именно сигары, и магазин их предлагает

>0.1

Итак, агент считает, что организация полностью релевантна, при этом оценку выдал 0.1. Это, конечно, ошибка даже не агента, а LLM, что-то переклинило в электрических мозгах. Но таких случаев немного, учитывая, что замена 0.1 на 1 повышает точность всего на 2%.

Другой случай - запрос `где в Астрахани сделать фото радужки глаза` для организации **Фото радужки глаза; ScreenEyes** по адресу **Владимир, Большая Московская улица, 16**. Агент считает, что это 0.0, хотя референсное значение 1. Но он прав:

> Организация ScreenEyes не имеет филиала в Астрахани согласно её официальному сайту. В списке городов есть 10 городов, но Астрахани нет. Адрес организации: г. Владимир, ул. Б. Московская 16.

> Пользователь явно ищет место в Астрахани, где можно сделать фото радужки глаза. Организация, находящаяся во Владимире, не может удовлетворить этот запрос, так как пользователю нужно физически посетить студию для съемки (это не онлайн-услуга).

> Таким образом, хотя организация идеально подходит по типу услуги, она полностью не соответствует по местоположению, что является важным требованием в запросе пользователя.

> 0.0

Посмотрим ещё пример. `мильгамма уколы купить в коврове` для организации **Столички; Stolichki в Коврове**. Референсное значение 0, агент считает, что организация релевантна.

> Отлично! Я нашел конкретную информацию. На странице видно, что в аптеке "Столички" по адресу "Ковров, Грибоедова, 28" (это именно та аптека, о которой идет речь) есть препарат "Мильгамма р-р д/ин. 2мл №5" по цене 561 рубль (на 22.01.26 09:48). Это подтверждает, что в данной аптеке есть препарат Мильгамма в форме уколов (раствор для инъекций).

> Теперь проанализируем запрос пользователя и организацию:

> 1. **Запрос пользователя**: "мильгамма уколы купить в коврове" - пользователь хочет купить препарат Мильгамма в форме уколов в городе Ковров.
2. **Организация**: Аптека "Столички" в Коврове по адресу ул. Грибоедова, 28.
3. **Соответствие**:
>- Организация находится в Коврове ✓
>- Это аптека, которая продает лекарства ✓
>- Конкретно в этой аптеке есть препарат Мильгамма в форме уколов (раствор для инъекций) ✓
4. **Важные требования**:
>- Покупка препарата Мильгамма - удовлетворено (препарат есть в наличии)
>- Форма уколов (инъекций) - удовлетворено (раствор для инъекций)
>- Место покупки - Ковров - удовлетворено (аптека находится в Коврове)
5. **Второстепенные требования**: В запросе нет других требований (например, конкретная цена, время работы и т.д.)

>Все важные требования пользователя удовлетворены. Организация полностью соответствует запросу пользователя.
>
>1.0

Как видно, агент опять прав. Ещё пример: `где поесть пельменей` для организации `Мархал`. Агент считает, что организация релевантна, референсное значение - 0.

> Отлично! Я нашел страницу, которая прямо указывает, что в кафе "Мархал" есть пельменная. На странице есть раздел "Пельменная в Кафе Мархал". Это означает, что кафе действительно предлагает пельмени как одну из своих услуг.

> Теперь проанализирую ситуацию:

> 1. **Запрос пользователя**: "где поесть пельменей" - пользователь ищет место, где можно поесть пельмени.
> 2. **Организация**: Кафе "Мархал" - кафе с кавказской и европейской кухней.
> 3. **Ключевой факт**: На официальном сайте кафе (marhal.obiz.ru) есть отдельная страница "Пельменная", что прямо указывает на то, что в кафе подают пельмени.
> 4. **Дополнительная информация**: В сводке цен из исходных данных пельмени не упоминаются, но это может быть связано с тем, что сводка неполная или устаревшая. Однако наличие отдельной страницы "Пельменная" на официальном сайте является убедительным доказательством.
> 5. **Важные требования**: Пользователь ищет место, где можно поесть пельмени. Кафе "Мархал" предлагает пельмени (согласно официальному сайту).
> 6. **Второстепенные требования**: Нет других указаний (тип заведения, цена, местоположение и т.д.).

> Таким образом, организация релевантна запросу пользователя, так как предлагает именно то, что он ищет - возможность поесть пельмени.

> 1.0

Вроде всё верно, на официальном сайте действительно явно и бессовестно указано, что они работают в том числе как пельменная. Но в меню пельменей нет, потому что реально это заведение кавказской кухни, и у них есть вместо этого хинкали. К сожалению, официальный сайт обманул нашего агента (хотя хинкали это почти пельмени, можно 0.1 поставить).

Ещё пример. `итальянский ресторан на ул строителей` для организации **Domino Pizza**. Агент считает, что организация нерелевантна.

`Теперь проанализирую все данные:

>1. **Запрос пользователя**: "итальянский ресторан на ул строителей"
>- Пользователь ищет итальянский ресторан
>- На улице Строителей

>2. **Информация об организации**:
>- Название: Domino Pizza (сеть пиццерий)
>- Адрес: Москва, проспект Вернадского, 29
>- Рубрика: Пиццерия
>- Это не итальянский ресторан, а пиццерия (сеть быстрого питания)

>3. **Что я нашел в поиске**:
>- На улице Строителей, 4, стр. 2 в Москве действительно есть итальянский ресторан "Andiamo" (Андиамо)
>- Это полноценный итальянский ресторан с итальянской кухней
>- Ресторан Andiamo соответствует запросу пользователя

>4. **Сравнение**:
>- Domino Pizza находится на проспекте Вернадского, 29, а не на улице Строителей
>- Domino Pizza - это пиццерия (сеть быстрого питания), а не итальянский ресторан
>- Пользователь ищет именно итальянский ресторан на улице Строителей

>5. **Важные требования пользователя**:
- Итальянский ресторан - НЕ удовлетворено (Domino Pizza - пиццерия, не итальянский ресторан)
- На улице Строителей - НЕ удовлетворено (адрес: проспект Вернадского, 29)

>**Вывод**: Организация Domino Pizza не соответствует запросу пользователя. Она не является итальянским рестораном и находится не на улице Строителей. Оба важных требования не удовлетворены.

>0.0

В этом случае агент опять прав. Domino Pizza действительно не является ни рестораном, ни находится на нужной улице.

И рассмотрим последний пример: `адрес налоговой инспекции по адресу проживания` для организации **Межрайонная ИФНС России № 2 по Самарской области** . Агент посчитал организацию нерелевантной запросу, хотя референсное значение 1.

>Теперь я понимаю, что для определения релевантности этой налоговой инспекции запросу пользователя, мне нужно понять следующее:

>1. Пользователь ищет "адрес налоговой инспекции по адресу проживания" - это означает, что ему нужна конкретная налоговая инспекция, которая обслуживает его место жительства.
2. Представленная организация - это Межрайонная ИФНС России № 2 по Самарской области, расположенная в Тольятти, бульвар Татищева, 12.
3. Эта инспекция обслуживает Автозаводский район Тольятти.

>Проблема в том, что у нас нет адреса проживания пользователя. Без этой информации мы не можем определить, является ли данная налоговая инспекция той самой, которая обслуживает адрес проживания пользователя.

>Однако, если рассматривать запрос в более широком смысле - пользователь хочет найти налоговую инспекцию, и мы предоставляем адрес налоговой инспекции. Но ключевое слово "по адресу проживания" указывает на то, что нужна не просто любая налоговая инспекция, а именно та, которая соответствует месту жительства пользователя.

>Поскольку у нас нет информации об адресе проживания пользователя, мы не можем утверждать, что эта конкретная налоговая инспекция соответствует его запросу. Пользователь мог бы использовать сервис ФНС для определения своей инспекции по адресу, но мы не знаем его адрес.

>В контексте поиска организации на карте, если пользователь указал "адрес налоговой инспекции по адресу проживания", это можно трактовать как поиск налоговой инспекции поблизости от его местоположения. Но опять же, без знания местоположения пользователя мы не можем определить релевантность.

>Проанализирую еще раз: организация является налоговой инспекцией (соответствует типу организации), имеет конкретный адрес (соответствует части запроса "адрес налоговой инспекции"), но не соответствует части "по адресу проживания", так как мы не знаем, совпадает ли адрес проживания пользователя с зоной обслуживания этой инспекции.

>Это важное требование - налоговая инспекция должна соответствовать адресу проживания. Поскольку мы не можем проверить это соответствие, организация не может считаться полностью релевантной.

>Оценка:
>- Важное требование: налоговая инспекция, соответствующая адресу проживания - НЕ ПОДТВЕРЖДЕНО (0.0)
>- Второстепенные требования: наличие адреса налоговой инспекции - удовлетворено

>Таким образом, организация нерелевантна запросу, так как не удовлетворено важное требование о соответствии адресу проживания.

>0.0

Формально, конечно, агент прав. Неизвестно, где проживает пользователь. Но мог бы и откинуть эту спорную и ненужную часть запроса.

**В итоге** - краткий обзор ошибок агента на валидационной выборке показывает, что он очень часто прав, несмотря на несоответствие референсному значению. Если он ошибается, то есть несколько причин для этого:
- ошибка LLM (в рассуждении пишет, что организация полностью релевантна, но ставит оценку 0.1)
- ошибка или вводящая в заблуждение неоднозначность на официальном сайте организации
- формулировка запроса, требующая уточнения для корректного ответа

Вероятно, с этими проблемами может помочь уточнение системного промта. Но это непростой итеративный процесс, когда улучшение в одном случае может сказаться в ухудшении в другом. Уже с нашим системным промтом модель почему-то начала путать 0.1 и 1.0. Кстати, с последним может помочь текстовая описание класса, потому что языковая модель путает слова меньше, чем цифры.

## Выводы

Таким образом, в этом проекте было сделано следующее:
1. мы создали baseline на основе LLM, добились точности 0.45 (на неисправленном датасете)
2. мы создали какого-то агента с яндекс-поиском (но его начальное состояние не было сохранено)
3. путем тестирования агента с поиском на тренировочном датасете был сформирован более детальный и точный системный промт, который существенно улучшил результаты агента
4. в некоторых примерах агенту было недостаточно информации от поиска, поэтому агент был расширен инструментом хождения по сайтам
5. на нескольких примерах было показано, что инструмент хождения по сайтам улучшает точность агента
6. итоговый агент со всеми инструментами был протестирован на валидационной выборке и показал существенно улучшенную точность 0.73
7. анализ расхождений в ответах агента и референсных значений показал, что весьма часто агент оказывается прав и требуется исправление референсных значений.
8. агента можно ещё улучшить с помощью уточнения системного промта